In [ ]:
import os
import matplotlib.pyplot    as plt
import numpy                as np

In [ ]:
def get_x_t(path):

    
    with open(path[1]) as file:
        lines = file.readlines()
        interpolated_dict = {}
        time = lines[0].split('T')[1]
        parts = time.split(':')
        char = '-'

        if ("+") in lines[0]:
            char = '+'
        
        min_t = float(parts[1]) * 60  + float(parts[0]) * 3600 #+ float(parts[2].split('-')[0]) 
        for line in lines:
            time = line.split('T')[1]
            x = float(line.split(',')[0])
            parts = time.split(':')
            t = float(parts[0]) * 3600 + float(parts[1]) * 60 + float(parts[2].split(char)[0]) - min_t
            interpolated_dict[x] = t

    with open(path[2]) as file:
        lines = file.readlines()
        real_dict = {}
        time = lines[0].split('T')[1]
        parts = time.split(':')
        min_t = float(parts[1]) * 60  + float(parts[0]) * 3600 #+ float(parts[2].split('-')[0]) 
        for line in lines:
            time = line.split('T')[1]
            x = float(line.split(',')[0])
            parts = time.split(':')
            t = float(parts[0]) * 3600 + float(parts[1]) * 60 + float(parts[2].split(char)[0]) - min_t
            real_dict[x] = t
    
    real_keys = list(real_dict.keys())
    real_values = list(real_dict.values())
 
    real_dict = {real_keys[i]: real_values[i] for i in range(len(real_keys) - 1) if real_keys[i + 1] >= real_keys[i]}
    
    print(f"{path[0]} loaded")
    return [path[0],real_dict, interpolated_dict]

In [ ]:
class FileParser:
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.box_folders = []

    def find_folders(self):
        for root, dirs, files in os.walk(self.root_dir):
                for dir_name in dirs:
                    if dir_name.startswith("Box1"):
                        adaptive_z_folder = os.path.join(root, dir_name, "AdaptiveZ_10mm")

                        if os.path.isdir(adaptive_z_folder): 
                            for part_folder in os.listdir(adaptive_z_folder):
                                full_part_path = os.path.join(adaptive_z_folder, part_folder)
                                if os.path.isdir(full_part_path) and "Part" in part_folder:
                                    fake_x_path = None
                                    real_x_path = None

                                    for file_name in os.listdir(full_part_path):
                                        if file_name.endswith("real.dbg"):
                                            real_x_path = os.path.join(full_part_path, file_name)
                                        elif file_name.endswith(".dbg"):
                                            fake_x_path = os.path.join(full_part_path, file_name)
                                   

                                    self.box_folders.append((part_folder,fake_x_path, real_x_path))

    def get_box_folders(self):
        self.find_folders()
        return self.box_folders


parser = FileParser(r"C:\Users\eashenhurst\Desktop\local macassa")
paths_list = parser.get_box_folders()

for name, fake_x_path,real_x_path  in paths_list:
    print(f"Part: {name}")
    if fake_x_path:
        print(f"  interpolated dbg: {fake_x_path}")
    if real_x_path:
        print(f"  real dbg: {real_x_path}")


In [ ]:
class FileParser:
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.box_folders = []

    def find_folders(self):
        for root, dirs, files in os.walk(self.root_dir):
                for dir_name in dirs:
                                full_part_path = os.path.join(root,dir_name)
                                
                                fake_x_path = None
                                real_x_path = None
                                for file_name in os.listdir(full_part_path):
                                    if file_name.endswith("real.dbg"):
                                        real_x_path = os.path.join(full_part_path, file_name)
                                    elif file_name.endswith(".dbg"):
                                        fake_x_path = os.path.join(full_part_path, file_name)
                                

                                self.box_folders.append((dir_name,fake_x_path, real_x_path))

    def get_box_folders(self):
        self.find_folders()
        return self.box_folders


parser = FileParser(r"\\192.168.1.63\g\CoreScans")
paths_list = parser.get_box_folders()

for name, fake_x_path,real_x_path  in paths_list:
    print(f"Part: {name}")
    print(f"  interpolated dbg: {fake_x_path}")
    print(f"  real dbg: {real_x_path}")

In [ ]:
position_dicts = []

for path in paths_list:
    if None not in path:
        position_dicts.append(get_x_t(path))

In [ ]:
def velocity_uncertainty(real_dict,fake_dict):

    dx_uncertainty = np.sqrt(0.005**2 + 0.005**2)
    dt_uncertainty = np.sqrt(0.008**2 + 0.008**2)
    
    real_positions = list(real_dict.keys())[::-1]
    real_times = list(real_dict.values())[::-1]
    lidar_positions= list(fake_dict.keys())[::-1]

    dt = list(np.diff(real_times))
    dx = list(np.diff(real_positions))
    dt.append(dt[-1])
    dx.append(dx[-1])

    dt = np.array(dt)
    dx = np.array(dx)

    uncertainties = np.sqrt((dx_uncertainty/dt)**2 +  ((dx/(dt**2))*dt_uncertainty)**2)

    uncertainties_interp = []

    for x in lidar_positions:
        if x <= np.max(real_positions):
            index = np.searchsorted(real_positions,x, side = 'right') - 1
            uncertainties_interp.append(uncertainties[index])


    return uncertainties_interp, uncertainties

In [ ]:
all_dt_real = []
all_dt_interp = []

for dict in position_dicts:
    x_real = list(dict[1].keys())
    t_real = list(dict[1].values())
    t_fake = list(dict[2].values())
    x_fake = list(dict[2].keys())
    dt_fake = np.diff(t_fake)
    v_real = np.abs(np.diff(x_fake)/dt_fake)
    v_real = v_real[200:-200]

    print(dict[0])
    vmean = np.nanmean(v_real)
    dtmean = np.nanmean(dt_fake)
    print(f"  {vmean}")
    #print(f"  {dtmean}")
    #print(f"  {(vmena*dtmean)}")



In [ ]:
import numpy as np
from scipy.signal import welch
import matplotlib.pyplot as plt

all_t_real = []
all_dt_real = []
all_t_interp = []
all_dt_interp = []

for dict in position_dicts:
    x_real = np.array(list(dict[1].keys()))
    t_real = np.array(list(dict[1].values()))
    dt_real = np.diff(t_real)
    
    frequencies, power = welch(1/dt_real, fs=1/np.mean(dt_real), nperseg=len(dt_real)//8)
    fundamental_freq = frequencies[np.argmax(power)]
    print(fundamental_freq)
    v_real = np.abs(np.diff(x_real) / dt_real)
    all_t_real.extend(t_real[:-1])
    all_dt_real.extend(dt_real)

    x_interp = np.array(list(dict[2].keys()))
    t_interp = np.array(list(dict[2].values()))
    dt_interp = np.diff(t_interp)
    frequencies, power = welch(1/dt_interp, fs=1/np.mean(dt_interp), nperseg=len(dt_interp)//8)
    fundamental_freq = frequencies[np.argmax(power)]
    print(fundamental_freq)
    v_interp = np.abs(np.diff(x_interp) / dt_interp)
    all_t_interp.extend(t_interp[:-1])
    all_dt_interp.extend(dt_interp)

fig = plt.figure(figsize=(70, 40), dpi=75)
gs = fig.add_gridspec(1, 2, hspace=0.025)
ax = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])

print(np.nanmean(all_dt_interp))

ax.scatter(all_t_real, all_dt_real, color="red", s=3)
ax2.scatter(all_t_interp, all_dt_interp, color="blue", s=3)

f = 2000

#y_values_32 = np.arange(2 / (f / 32), 29 / (f / 32), 1 / (f / 32))
#ax.hlines(y=y_values_32, xmin=min(all_t_real), xmax=max(all_t_real), color='green', linestyle='solid', linewidth=1)

#y_values_96 = np.arange(0, 10 / (f / 96), 1 / (f / 96))
#ax.hlines(y=y_values_96, xmin=min(all_t_real), xmax=max(all_t_real), color='red', linestyle='solid', linewidth=1)

ax.axhline(y=1/1.6/4, color='purple', linestyle='solid', linewidth=2)

#y_values_f = np.arange(0, 100 / f, 1 / f)
#ax2.hlines(y=y_values_f, xmin=min(all_t_interp), xmax=max(all_t_interp), color='green', linestyle='solid', linewidth=1)

ax.set_ybound(0, 0.5)
ax.set_yticks(np.arange(0, 0.5, 0.01))
ax2.set_ybound(0.01, 0.03)
ax2.set_yticks(np.arange(0.01, 0.031, 0.001))

ax.set_title("Table Coordinate delta ts", fontsize=20)
ax2.set_title("Lidar reading delta ts", fontsize=20)

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

all_dt_real = []
all_dt_interp = []

for dict in position_dicts:
    x_real = np.array(list(dict[1].keys()))
    t_real = np.array(list(dict[1].values()))
    dt_real = np.diff(t_real)
    all_dt_real.extend(dt_real)

    x_interp = np.array(list(dict[2].keys()))
    t_interp = np.array(list(dict[2].values()))
    dt_interp = np.diff(t_interp)
    all_dt_interp.extend(dt_interp)

fig = plt.figure(figsize=(70, 15), dpi=150)
gs = fig.add_gridspec(1, 2, hspace=0.025)
ax = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])

bin_width = 0.001
bins_real = np.arange(0, max(all_dt_real) + bin_width, 0.001)
bins_interp = np.arange(0, max(all_dt_interp) + 0.002, bin_width)

# Create histograms
counts_real, _ = np.histogram(all_dt_real, bins=bins_real)
counts_interp, _ = np.histogram(all_dt_interp, bins=bins_interp)

ax.bar(bins_real[:-1], counts_real, width=np.diff(bins_real), color="red")

f = 2000
oset = 0
x = np.linspace(0, (0.5), 100000)  
s = 10
y = ((np.sin(2 * np.pi * 0.84 * (x))))*10000# * np.sin(2 * np.pi * f/96/2 * (x + oset))*np.sin(2 * np.pi * 1/0.224 * (x)))*5000 

#ax.plot(x, y, color='black', linewidth=1)

ax2.bar(bins_interp[:-1], counts_interp, width=np.diff(bins_interp), color="blue")
ax2.axvline(x = 0.01996, linewidth = 0.0021, color = 'red')

ax.set_xbound(0, 0.075)
ax2.set_xbound(0, 0.1)
ax.set_title("Table Coordinate delta ts", fontsize=20)
ax2.set_title("Lidar reading delta ts", fontsize=20)

plt.show()


In [ ]:
fig = plt.figure(figsize=(100,10) , dpi=150)
gs = fig.add_gridspec(1, 1, hspace=0.025)

ax = fig.add_subplot(gs[0,0])
dict =  position_dicts[0]



x_real = list(dict[1].keys())
t_real = list(dict[1].values())
dt_real = np.diff(t_real)
v_real = np.abs(np.diff(x_real)/dt_real)
t_real = t_real[:-1]


x_interp = list(dict[2].keys())
t_interp = list(dict[2].values())
dt_interp = np.diff(t_interp)
v_interp = np.abs(np.diff(x_interp)/dt_interp)
t_interp = t_interp[:-1]

unc_interp = list(velocity_uncertainty(dict[1],dict[2])[0])[::-1]
unc_real = list(velocity_uncertainty(dict[1],dict[2])[1])[::-1]

while len(unc_interp) > len(v_interp):
    unc_interp = unc_interp[1:]

while len(unc_interp) < len(v_interp):
    unc_interp.append(unc_interp[-1])

unc_real = unc_real[1:]

t_interp = np.array(t_interp)
t_real = np.array(t_real)


ax.axhline(y= 50, color = 'cyan',linestyle = 'solid', linewidth = 10)

#ax.vlines(t_interp,ymin = v_interp - unc_interp, ymax = v_interp + unc_interp, color = "blue", alpha = 0.5)
ax.scatter(t_interp,v_interp - unc_interp,color = "green", alpha = 1, s = 1)
ax.scatter(t_interp,v_interp + unc_interp,color = "green", alpha = 1, s = 1)
ax.scatter(t_interp,v_interp,color = "blue", s = 2.5)
#ax.vlines(t_real,ymin = v_real - unc_real, ymax = v_real + unc_real, color = "red", alpha = 1, linewidth = 0.5)
ax.scatter(t_real,v_real, color = "Red", s = 7)
ax.set_ylim(40, 60)
ax.set_xticks(np.arange(min(t_interp), max(t_interp) + 1, 0.25))

#f = 0.840
#oset = 0
#x = np.linspace(np.nanmin(t_interp), np.nanmax(t_interp), 100000)  
#y = np.sin(2 * np.pi * f * (x + oset))  + 50         
#ax.plot(x, y, color='black', linewidth=1)

ax.set_title(dict[0], fontsize=20)

plt.show()

In [ ]:
start = 0
plot_range = 20

fig = plt.figure(figsize=(90,8 * 25), dpi=75)
gs = fig.add_gridspec(25, 1, hspace=0.025)

for i in range(len(position_dicts)):
#for i in range(25):
        dict = position_dicts[i - start]
        ax = fig.add_subplot(gs[i - start,0])

        x_real = list(dict[1].keys())
        t_real = list(dict[1].values())
        dt_real = np.diff(t_real)
        v_real = np.abs(np.diff(x_real)/dt_real)
        t_real = t_real[:-1]


        x_interp = list(dict[2].keys())
        t_interp = list(dict[2].values())
        dt_interp = np.diff(t_interp)
        v_interp = np.abs(np.diff(x_interp)/dt_interp)
        t_interp = t_interp[:-1]

        unc_interp = list(velocity_uncertainty(dict[1],dict[2])[0])[::-1]
        unc_real = list(velocity_uncertainty(dict[1],dict[2])[1])[::-1]

        while len(unc_interp) > len(v_interp):
            unc_interp = unc_interp[1:]

        while len(unc_interp) < len(v_interp):
            unc_interp.append(unc_interp[-1])

        unc_real = unc_real[1:]

        t_interp = np.array(t_interp)
        t_real = np.array(t_real)

        ax.axhline(y= 50, color = 'cyan',linestyle = 'solid', linewidth = 10)

        #ax.vlines(t_interp,ymin = v_interp - unc_interp, ymax = v_interp + unc_interp, color = "blue", alpha = 0.5)
        #ax.scatter(t_interp,v_interp - unc_interp,color = "green", alpha = 1, s = 1)
        #ax.scatter(t_interp,v_interp + unc_interp,color = "green", alpha = 1, s = 1)
        #ax.scatter(t_interp,v_interp,color = "blue", s = 2.5)
        #ax.vlines(t_real,ymin = v_real - unc_real, ymax = v_real + unc_real, color = "red", alpha = 1, linewidth = 0.5)
        ax.scatter(t_interp,dt_interp % 0.002, color = "Red", s = 7)
        ax.set_ylim(0,0.003)

        f = 4.4642857
        oset = 0.5
        x = np.linspace(np.nanmin(t_interp), np.nanmax(t_interp), 10000)  
        y = np.sin(2 * np.pi * f * (x + oset))  + 50         
        ax.plot(x, y, color='black', linewidth=1)

        ax.set_title(dict[0], fontsize=20)

plt.show()

In [ ]:


fig = plt.figure(figsize=(90,20), dpi=75)
gs = fig.add_gridspec(1, 1, hspace=0.025)

all_dt_real = []
all_a_real = []
ax = fig.add_subplot(gs[0,0])

for i in range(len(position_dicts)):
    dict = position_dicts[i]

    x_real = list(dict[1].keys())
    t_real = np.array(list(dict[1].values()))

    dt_real = np.diff(t_real)
    
    v_real = np.abs(np.diff(x_real)/dt_real)
    error = list(50 - v_real)
    error = error[1:]
    t_velocity = (t_real[:-1] + t_real[1:]) / 2
    dt_v = np.diff(t_velocity)
    a_real = np.diff(v_real)/dt_v
    t_a = (t_velocity[:-1] + t_velocity[1:]) / 2
    dt_a = np.diff(t_a)
  

    all_dt_real.extend(error)
    all_a_real.extend(a_real)
           
plt.scatter(all_dt_real,all_a_real)

plt.show()

In [ ]:
start = 0
plot_range = len(position_dicts)
from scipy.interpolate import make_interp_spline
from scipy.interpolate import CubicSpline
from scipy.interpolate import UnivariateSpline

#fig = plt.figure(figsize=(90,8 * len(position_dicts)), dpi=75)
#gs = fig.add_gridspec(len(position_dicts), 1, hspace=0.025)

fig = plt.figure(figsize=(90,8 * plot_range), dpi=75)
gs = fig.add_gridspec(plot_range, 1, hspace=0.025)

for i in range(start,plot_range+start):
        dict = position_dicts[i - start]
        ax = fig.add_subplot(gs[i - start,0])

        x = list(dict[1].keys())
        t = np.array(list(dict[1].values()))
        dt = np.diff(t)

        v = np.abs(np.diff(x)/dt)
        t_v = (t[:-1] + t[1:]) / 2
        dt_v = np.diff(t_v)

        e = (v- 50)*10

        de = []  
        ie = [0] 

        for i in range(1, len(e)):
                de.append((e[i] - e[i-1])/dt_v[i-1])

                ie.append(ie[-1] + e[i] * dt_v[i-1])

        e=np.array(e)
        de=np.array(de)/5
        ie=np.array(ie)

     
        a = np.array(np.diff(v)/dt_v)
        t_a = (t_v[:-1] + t_v[1:]) / 2
        dt_a = np.diff(t_a)

        j = np.array(np.diff(a)/dt_a)
        t_j = (t_a[:-1] + t_a[1:]) / 2


        #ax.scatter(t_a,de, color = "Red", s = 100)
        #ax.scatter(t_v,(e + de + ie), color = "black", s = 100)

        ax.scatter(t_v,v, color = "blue", s = 100)
        #ax.scatter(t_j,j, color = "purple", s = 100)
        #ax.axhline(y = 0)

        max_slope = 10
        bc_type = ((1, max_slope), (1, max_slope))
       
        x_smooth = np.linspace(min(t_v), max(t_v), 10000)
        #spl1 = UnivariateSpline(t_v, v, s = max_slope)
        spl2 = make_interp_spline(t, x, k=5)  
        #spl3 = make_interp_spline(t_a,de, k=5)  
        #spl4 = make_interp_spline(t_v,ie, k=5) 
        #spl5 = make_interp_spline(t_v,(de + e + ie), k=5)  

        #y_smooth1 = spl1(x_smooth)

        y_smooth2 = spl2(x_smooth)  
        #y_smooth3 = spl3(x_smooth)
        #y_smooth4 = spl4(x_smooth) 
        #y_smooth5 = spl5(x_smooth) 

        #ax.plot(x_smooth, y_smooth1, color="red")
        ax.plot(x_smooth, y_smooth2,alpha = 0.1, color="blue")
        #ax.plot(x_smooth, y_smooth3,alpha = 0.1, color="green")
        #ax.plot(x_smooth, y_smooth4,alpha = 0.1, color="purple")
        #ax.plot(x_smooth, y_smooth3 + y_smooth2,alpha = 1, color="black")
        #ax.plot(x_smooth, y_smooth5,alpha = 1, color="black")
        ax.set_title(dict[0], fontsize=20)



plt.show()